In [76]:
import time
import random
import math

In [77]:
def read_tsp_file(filename):
    with open(filename, 'r') as f:
        first_line = f.readline().strip()
        n = int(first_line)
        header = f.readline()
        dist = [[0.0 if i == j else math.inf for j in range(n)] for i in range(n)]

        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) != 3:
                continue  
            i, j, d = parts
            i = int(i) - 1   
            j = int(j) - 1
            d = float(d)
            dist[i][j] = d
            dist[j][i] = d   

    return n, dist

In [78]:
# 1000 Node graph files:
file_euclid = r"C:\Users\humza\Downloads\TSP_1000_euclidianDistance (1).txt"
file_random = r"C:\Users\humza\Downloads\TSP_1000_randomDistance (1).txt"

In [79]:
def path_length(path, dist):
    total = 0.0
    n = len(path)
    for i in range(n):
        total += dist[path[i]][path[(i+1)%n]]
    return total

def nearest_neighbor_path(n, dist, start=None):
    if start is None:
        start = random.randrange(n)

    unvisited = set(range(n))
    unvisited.remove(start)
    path = [start]
    current_node = start

    while unvisited:
        next_node = min(unvisited, key=lambda j: dist[current_node][j])
        path.append(next_node)
        unvisited.remove(next_node)
        current_node = next_node

    return path

In [80]:
def two_opt(path, dist, time_limit, start_time, eval_counter):
    n = len(path)
    best_path = path[:]
    best_length = path_length(best_path, dist)
    eval_counter["count"] += 1

    improved = True
    while improved and (time.time() - start_time < time_limit):
        improved = False
        for i in range(1, n-2):
            if time.time() - start_time >= time_limit:
                break
            for k in range(i+1, n-1):
                if time.time() - start_time >= time_limit:
                    break

                a, b = best_path[i-1], best_path[i]
                c, d = best_path[k], best_path[(k+1) % n]

                old_cost = dist[a][b] + dist[c][d]
                new_cost = dist[a][c] + dist[b][d]
                eval_counter["count"] += 1

                if new_cost < old_cost:
                    best_path[i:k+1] = reversed(best_path[i:k+1])
                    best_length = best_length - old_cost + new_cost
                    improved = True
                    break
            if improved:
                break

    return best_path, best_length

In [81]:
def solve_tsp_instance(filename, time_limit_seconds=50, seed=0):
    random.seed(seed)

    n, dist = read_tsp_file(filename)

    start_time = time.time()
    eval_counter = {"count": 0}

    best_path = nearest_neighbor_path(n, dist)
    best_length = path_length(best_path, dist)
    eval_counter["count"] += 1

    best_path, best_length = two_opt(best_path, dist, time_limit_seconds, start_time, eval_counter)

    path_1based = [x+1 for x in best_path]
    path_1based.append(path_1based[0])

    return path_1based, best_length, eval_counter["count"]

In [82]:
print("Solving Euclidean...")
path_e, cost_e, eval_e = solve_tsp_instance(file_euclid)
print("Cost:", round(cost_e, 2))
print("Number of Cycles:", eval_e)
print("Number of Cycle (scientific notation):", f"{eval_e:.0e}")


print("\nSolving Random...")
path_r, cost_r, eval_r = solve_tsp_instance(file_random)
print("Cost:", round(cost_r, 2))
print("Number of Cycles:", eval_r)
print("Number of Cycles (scientific notation):", f"{eval_r:.0e}")

Solving Euclidean...
Cost: 2712.0
Number of Cycles: 63313623
Number of Cycle (scientific notation): 6e+07

Solving Random...
Cost: 420.12
Number of Cycles: 9870865
Number of Cycles (scientific notation): 1e+07


In [83]:
SID = "925628952"

In [85]:
def save_paths_to_file(path_e_1based, path_r_1based, filename):
    with open(filename, 'w') as f:
        
        line_e = ", ".join(str(v) for v in path_e_1based)
        f.write(line_e + "\n")

        
        line_r = ", ".join(str(v) for v in path_r_1based)
        f.write(line_r + "\n")

output_filename = fr"C:\Users\humza\Downloads\solution_{SID}.txt"
save_paths_to_file(path_e, path_r, output_filename)

print("Saved solution file:")
print(f"solution_{SID}.txt")


Saved solution file:
solution_925628952.txt
